extract_t2m_avg_monthly.ipynb: extract average monthly temperature at 2m elevation for each month for every data point in contiguous US that gro-intelligence provides crop yield for


In [1]:
import xarray as xr
import os
import pandas as pd

In [ ]:
# path to annual wheat yields data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/gro_ylds/wheatf_mn_ia.xlsx'
y = pd.read_excel(path)

In [ ]:
# path to monthly t2m data
path = '/Users/demetrayancopoulos/Desktop/SML310/final_project/era5/monthly_t2m.nc'
dsm = xr.open_dataset(path)

Round to the nearest 1/4:

https://stackoverflow.com/questions/33257635/python-round-to-nearest-0-25

In [7]:
def x_rnd(x):
    return round(x*4)/4

In [20]:
cols = ['yr', 'lat', 'lon', 'jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
temps = pd.DataFrame(index = range(0, len(y.index)), columns=cols)

for ix in y.index:
    
    lat = y.loc[ix].lat     # retrieve latitude at this entry
    lat = x_rnd(lat)       # round lat coord to nearest 0.25
    
    lon = y.loc[ix].lon     # retrive longitude at this entry
    lon = x_rnd(lon)        # round lon coord to nearest 0.25
    
    yr = y.loc[ix].StartDate[0] + y.loc[ix].StartDate[1] + y.loc[ix].StartDate[2] + y.loc[ix].StartDate[3]
    
    # t2m at this lat/lon/yr
    t2m = dsm.t2m.sel(latitude=lat, longitude=lon)
    
    # store monthly average t2m for this lat/lon/yr
    for i in range(1, 13):
        t = yr + '-' + str(i)
        temps.iat[ix, i+2] = t2m.sel(time=t).values[0]
    
    # store yr for this point
    col = 0
    temps.iat[ix, col] = yr
    
    # store latitude for this point
    col = 1
    temps.iat[ix, col] = lat
    
    # store longitude for this point
    col = 2
    temps.iat[ix, col] = lon
